In [14]:
import open3d as o3d
import numpy as np
import matplotlib.cm as plt
import math
import time

pcd_dir = "../extracted_pcl/tower/"

In [2]:
def readFile(FileName):
    path = pcd_dir + FileName
    print("Reading in file")
    pcd = o3d.io.read_point_cloud(path)
    print(pcd.get_min_bound())
    print(pcd.get_max_bound())
    print(pcd)
    # print(np.asarray(pcd.points))
    xyz = np.asarray(pcd.points)
    xyz = xyz[xyz[:, 2] > 0.2]
    pcd.points = o3d.utility.Vector3dVector(xyz)

    # plane_model, inliers = pcd.segment_plane(distance_threshold=0.1,
    #                                      ransac_n=3,
    #                                      num_iterations=1000)
    # [a, b, c, d] = plane_model
    # print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

    # pcd = pcd.select_by_index(inliers, True)

    with o3d.utility.VerbosityContextManager(
            o3d.utility.VerbosityLevel.Debug) as cm:
        labels = np.array(
            pcd.cluster_dbscan(eps=1, min_points=5, print_progress=True))

    max_label = labels.max()
    values, counts = np.unique(labels, return_counts=True)
    idxs = np.where(labels == values[counts.argmax()])[0]
    print(idxs)
    pcd = pcd.select_by_index(idxs)

    print(f"point cloud has {max_label + 1} clusters")
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0
    pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

    # o3d.visualization.draw_geometries([pcd])
    # alpha = 0.5
    # print(f"alpha={alpha:.3f}")
    # mesh, pt_map = o3d.geometry.TetraMesh.create_from_point_cloud(pcd)

    # mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha, tetra_mesh, pt_map)
    # mesh.compute_vertex_normals()
    # o3d.visualization.draw_geometries([pcd, mesh], mesh_show_back_face=True)
    # with o3d.utility.VerbosityContextManager(
    #     o3d.utility.VerbosityLevel.Debug) as cm:
    #     labels = np.array(
    #         pcd.cluster_dbscan(eps=0.02, min_points=10, print_progress=True))
    # o3d.visualization.draw_geometries([mesh])

    # max_label = labels.max()
    # print(f"point cloud has {max_label + 1} clusters")
    # colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
    # colors[labels < 0] = 0
    # pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
    # o3d.visualization.draw_geometries([pcd],
    #                                 zoom=0.455,
    #                                 front=[-0.4999, -0.1659, -0.8499],
    #                                 lookat=[2.1813, 2.0619, 2.0999],
    #                                 up=[0.1204, -0.9852, 0.1215])
    # pcl, ind = pcd.remove_radius_outlier(nb_points=30, radius=1)

    # hull, _ = pcd.compute_convex_hull()
    # hull_ls = o3d.geometry.LineSet.create_from_triangle_mesh(hull)
    # hull_ls.paint_uniform_color((1, 0, 0))
    # o3d.visualization.draw_geometries([pcd, hull_ls])
    return o3d.io.read_point_cloud(path), pcd


In [3]:
def extract_properties(pcd_in):
    xyz = np.asarray(pcd_in.points)

    base = xyz[xyz[:, 2] < 2]
    base_pcd = o3d.geometry.PointCloud()
    base_pcd.points = o3d.utility.Vector3dVector(base)
    obb = base_pcd.get_axis_aligned_bounding_box()
    center = obb.get_center()
    center[2] = 0.3
    extent = obb.get_extent()
    width = min(extent[0], extent[1])
    height = pcd_in.get_max_bound()[2]
    print(width)
    print(center)

    plane_model, inliers = pcd_in.segment_plane(distance_threshold=0.3,
                                         ransac_n=3,
                                         num_iterations=1000)
    [a, b, c, d] = plane_model
    print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

    pcd = pcd_in.select_by_index(inliers)
    # o3d.visualization.draw_geometries([pcd])
    extent = pcd.get_axis_aligned_bounding_box().get_extent()
    slope = min(abs(extent[0]/extent[2]),abs(extent[1]/extent[2]))
    print(slope)
    return center, width, 0, height, slope

In [4]:
pcd_file = "973300000.pcd"
orig, pcd = readFile(pcd_file); # Filtered pointcloud without noise or ground
orig.translate([20,0,0])
extract_properties(pcd)
# o3d.visualization.draw_geometries([orig,pcd])

Reading in file
[ 10.1      -34.900002   0.1     ]
[ 34.900002 -15.1       48.099998]
PointCloud with 25298 points.
[Open3D DEBUG] Precompute Neighbours
[Open3D DEBUG] Done Precompute Neighbours
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 23
[    0     1     2 ... 20049 20050 20051]
point cloud has 23 clusters
9.799999
[ 19.9       -24.6000005   0.3      ]
Plane equation: 1.00x + 0.00y + 0.09z + -24.63 = 0
0.09905657787580155


(array([ 19.9      , -24.6000005,   0.3      ]),
 9.799999,
 0,
 44.099998,
 0.09905657787580155)

In [24]:
def construct_pyramid_pcd(center, width, orientation, height, slope):
    top_width = width - 2*(slope*height)
    base_xyz =  [[center[0]+width/2, center[1]+width/2, center[2]],
                [center[0]+width/2, center[1]-width/2, center[2]],
                [center[0]-width/2, center[1]+width/2, center[2]],
                [center[0]-width/2, center[1]-width/2, center[2]]]
    top_xyz =   [[center[0]+top_width/2, center[1]+top_width/2, height],
                [center[0]+top_width/2, center[1]-top_width/2, height],
                [center[0]-top_width/2, center[1]+top_width/2, height],
                [center[0]-top_width/2, center[1]-top_width/2, height]]
    points = np.concatenate((np.asarray(base_xyz), np.asarray(top_xyz)),axis=0)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    mesh = pcd.compute_convex_hull()
    # alpha = 0.03
    # print(f"alpha={alpha:.3f}")
    # mesh = o3d.geometry.TriangleMesh.create_from_point_cloud(pcd)
    # mesh.compute_vertex_normals()
    # o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)
    # o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)
    # print(mesh)
    pcd = mesh[0].sample_points_poisson_disk(number_of_points=2000, init_factor=5)
    xyz = np.asarray(pcd.points)
    pcd.points = o3d.utility.Vector3dVector(xyz[xyz[:,2] > 0.3])
    return pcd, mesh

def construct_cone_pcd(center, width, orientation, height):
    mesh = o3d.geometry.TriangleMesh.create_cone(math.sqrt(2)*width/2, height)
    mesh.translate(center)
    pcd = mesh.sample_points_poisson_disk(number_of_points=2000, init_factor=5)
    xyz = np.asarray(pcd.points)
    pcd.points = o3d.utility.Vector3dVector(xyz[xyz[:,2] > 0.3])
    return pcd, mesh



In [22]:
pcd_file = "973300000.pcd"
start = time.time()
orig, pcd = readFile(pcd_file); # Filtered pointcloud without noise or ground
center, width, position, height, slope = extract_properties(pcd)
mesh, meshmesh = construct_cone_pcd(center, width, True, height)
# orig.translate([40,0,0])
# temp = o3d.geometry.PointCloud()
# temp.points = pcd.points
# temp.translate([20,0,0])
# o3d.visualization.draw_geometries([pcd,meshmesh], mesh_show_back_face=True)
print("Initial alignment")
evaluation = o3d.pipelines.registration.evaluate_registration(
    mesh, pcd, 0.5)
print(str(evaluation.fitness) + "   " + str(evaluation.inlier_rmse))
end = time.time()
print(end-start)

Reading in file
[ 10.1      -34.900002   0.1     ]
[ 34.900002 -15.1       48.099998]
PointCloud with 25298 points.
[Open3D DEBUG] Precompute Neighbours
[Open3D DEBUG] Done Precompute Neighbours
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 23
[    0     1     2 ... 20049 20050 20051]
point cloud has 23 clusters
9.799999
[ 19.9       -24.6000005   0.3      ]
Plane equation: 1.00x + 0.01y + 0.08z + -24.39 = 0
0.09302325799891838
Initial alignment
0.36220472440944884   0.2940178489735888
0.7418951988220215


In [26]:
pcd_file = "973300000.pcd"
start = time.time()
orig, pcd = readFile(pcd_file); # Filtered pointcloud without noise or ground
center, width, orientation, height, slope = extract_properties(pcd)
mesh, meshmesh = construct_pyramid_pcd(center, width, orientation, height, slope)
# orig.translate([40,0,0])
# temp = o3d.geometry.PointCloud()
# temp.points = pcd.points
# temp.translate([20,0,0])
# o3d.visualization.draw_geometries([pcd,meshmesh], mesh_show_back_face=True)
print("Initial alignment")
evaluation = o3d.pipelines.registration.evaluate_registration(
    mesh, pcd, 0.5)
print(str(evaluation.fitness) + "   " + str(evaluation.inlier_rmse))
end = time.time()
print(end-start)

Reading in file
[ 10.1      -34.900002   0.1     ]
[ 34.900002 -15.1       48.099998]
PointCloud with 25298 points.
[Open3D DEBUG] Precompute Neighbours
[Open3D DEBUG] Done Precompute Neighbours
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 23
[    0     1     2 ... 20049 20050 20051]
point cloud has 23 clusters
9.799999
[ 19.9       -24.6000005   0.3      ]
Plane equation: -0.01x + 1.00y + 0.08z + 19.88 = 0
0.08675797201788867
Initial alignment
0.6666666666666666   0.2527494858761624
0.7584686279296875


In [8]:
o3d.visualization.draw_geometries([mesh])

In [9]:
print("Initial alignment")
evaluation = o3d.pipelines.registration.evaluate_registration(
    mesh, pcd, 0.5)
print(evaluation)

Initial alignment
RegistrationResult with fitness=4.179188e-01, inlier_rmse=2.681977e-01, and correspondence_set size of 751
Access transformation to get result.


In [10]:
pcd_file = "973300000.pcd"
orig1, pcd1 = readFile(pcd_file); # Filtered pointcloud without noise or ground
pcd1.translate([-50,0,0])

pcd_file = "94856000.pcd"
orig2, pcd2 = readFile(pcd_file); # Filtered pointcloud without noise or ground

pcd_file = "816172000.pcd"
orig3, pcd3 = readFile(pcd_file); # Filtered pointcloud without noise or ground
pcd3.translate([-25,0,0])

o3d.visualization.draw_geometries([pcd1,pcd2,pcd3])

Reading in file
[ 10.1      -34.900002   0.1     ]
[ 34.900002 -15.1       48.099998]
PointCloud with 25298 points.
[Open3D DEBUG] Precompute Neighbours
[Open3D DEBUG] Done Precompute Neighbours
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 23
[    0     1     2 ... 20049 20050 20051]
point cloud has 23 clusters
Reading in file
[ 10.1 -31.9   0.1]
[ 28.299999 -15.1       24.5     ]
PointCloud with 3763 points.
[Open3D DEBUG] Precompute Neighbours
[Open3D DEBUG] Done Precompute Neighbours
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 3
[   0    1    2 ... 3017 3018 3019]
point cloud has 3 clusters
Reading in file
[ 10.1      -34.900002   0.1     ]
[ 34.900002 -15.1       32.900002]
PointCloud with 16420 points.
[Open3D DEBUG] Precompute Neighbours
[Open3D DEBUG] Done Precompute Neighbours
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 6
[    0     1     2 ... 11579 11580 11581]
point cloud has 6 clusters
